# Activity Recognition on Sagemaker

This notebook contains commands that train and deploy a model on Sagemaker for the activity recognition project mentioned in the README of this repository. In the below cells, we take the most accurate model from the ActivityRecognition.ipynb notebook and deploy it in a Sagemaker environment. 

With the massive increase in data collection and availability, the days of developing a machine learning model on a local Jupyter notebook are numbered. For this reason, ML engineers and data scientists will be looking to leverage the cloud for running their workloads. 

Training and deploying our model in the cloud has several benefits. First, we can take advantage of AWS' on-demand resources and pay-as-you-go model. We only have to provision resources while we need them for either training or inference and do not have to pay for those resources while we are not using them. AWS provides many options for instance types, including both CPU and GPU instances. Instead of having to procure expensive hardware, we can run our workload on these instances and only pay for the amount of time that we are running our workload. Another benefit of moving our ML workload to the cloud is that we can experiment quickly. With access to virtually unlimited hardware and the ability to quickly spin up instances and other resources, the cloud is a good fit for most ML jobs. The benefits are more easily seen with training jobs that are compute-intensive. Our simple project may not benefit from moving to the cloud as much as training a distributed neural network on several GB's of data, for example. However, this notebook provides a starting point for moving your local ML workloads to the cloud and demonstrates my ability to leverage AWS for machine learning. 

There are three main options for using Sagemaker - use a built-in algorithm, use a pre-built container (script mode), or bring-your-own container. Since we want to use some of the most common ML frameworks while still writing our own custom logic, we will use the a pre-built container. We will first aggregate our code from our local notebook into a single Python script. Sagemaker uses Amazon Simple Storage Service (S3) as its datasource so we will need to upload our data to S3 using the AWS SDK. We will then use the applicable framework container provided by Sagemaker to run our script and pull our training data from S3. Once the model is trained, we will deploy it to an inference endpoint, where we can send new data points and obtain a prediction. Our notebook will also include a step to cleanup our environment (delete our endpoint) so that we are not paying for unnecessary resources. 

In [ ]:
TODO:

Convert most accurate model into a Python script

Create notebook with commands to run script mode on Sagemaker and run it

Deploy model to endpoint

Run inference on endpoint

Destroy endpoint programmatically 